In [ ]:
package_paths = ['../input/imgclsmob/imgclsmob-master', 
                '../input/efficientnetpytorch/EfficientNet-PyTorch-master', 
                '../input/timm-pytorch-image-models/pytorch-image-models-master']
import sys
for package_path in package_paths:
    sys.path.append(package_path)

In [ ]:
import numpy as np
import pandas as pd
import os
import torch
import random
from albumentations import *
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
import timm
import torch.nn as nn
import torch.nn.functional as F
from efficientnet_pytorch import EfficientNet
from pytorch.pytorchcv.model_provider import get_model as ptcv_get_model
from tqdm import tqdm

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
class CasDataset(Dataset):

    def __init__(self, df, path, transforms):
        self.df = df
        self.path = path
        self.transforms = transforms

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idx):
        image = cv2.imread(self.path+self.df.loc[idx, 'image_id'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms:
            transformed = self.transforms(image=image)
            image = transformed['image']

        return image

In [ ]:
class CasModel(nn.Module):

    def __init__(self, num_classes=5, model="resnet34"):
        super().__init__()

        if model == "resnet16":
            self.backbone = ptcv_get_model("resnet16", pretrained=False)
            self.backbone.features.final_pool = nn.AdaptiveAvgPool2d(1)
            self.backbone.output = nn.Sequential(nn.Linear(512, num_classes))
        elif model == "resnet34":
            self.backbone = ptcv_get_model("resnet34", pretrained=False)
            self.backbone.features.final_pool = nn.AdaptiveAvgPool2d(1)
            self.backbone.output = nn.Sequential(nn.Linear(512, num_classes))
        elif model == "efficientnet-b2":
            self.backbone = EfficientNet.from_pretrained("efficientnet-b2")
            self.backbone._fc = nn.Linear(self.backbone._fc.in_features, num_classes)
        elif model == "efficientnet-b4":
            self.backbone = EfficientNet.from_pretrained("efficientnet-b4")
            self.backbone._fc = nn.Linear(self.backbone._fc.in_features, num_classes)
        elif model == "efficientnet-b7":
            self.backbone = EfficientNet.from_pretrained("efficientnet-b7")
            self.backbone._fc = nn.Linear(self.backbone._fc.in_features, num_classes)
        elif model == "vit-b":
            self.backbone = timm.create_model("vit_base_patch16_384", pretrained=False)
            self.backbone.head = nn.Linear(self.backbone.head.in_features, num_classes)
        elif model == "vit-l":
            self.backbone = timm.create_model("vit_large_patch16_384", pretrained=False)
            self.backbone.head = nn.Linear(self.backbone.head.in_features, num_classes)
        elif model == "vit-r":
            self.backbone = timm.create_model("vit_base_resnet50d_224", pretrained=False)
            self.backbone.head = nn.Linear(self.backbone.head.in_features, num_classes)
        elif model == "efficientnet-b4-timm":
            self.backbone = timm.create_model("tf_efficientnet_b4_ns", pretrained=False)
            self.backbone.classifier = nn.Linear(self.backbone.classifier.in_features, num_classes)
        else:
            raise NotImplementedError

    def forward(self, x):
        x = self.backbone(x)
        # x = F.sigmoid(x)

        return x

In [ ]:
def inference_one_epoch(model, data_loader, device):
    model.eval()

    image_preds_all = []
    
    for step, (imgs) in enumerate(data_loader):
        imgs = imgs.to(device).float()
        
        image_preds = model(imgs)   #output = model(input)
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)
    return image_preds_all

In [ ]:
tta = 12
IMAGE_SIZE = 512

if tta > 1:
    transforms_test = Compose([
            CenterCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
#             Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
#             Resize(IMAGE_SIZE, IMAGE_SIZE, p=1.),
            HorizontalFlip(p=0.5),
            VerticalFlip(p=0.5),
            Transpose(p=0.5),
#             RandomBrightnessContrast(brightness_limit=(-0.1, 0.1),
#                                      contrast_limit=(-0.1, 0.1),
#                                      p=0.5),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
            ToTensorV2(p=1.0),
        ])
else:
    transforms_test = Compose([
            CenterCrop(IMAGE_SIZE, IMAGE_SIZE, p=1.0),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
                max_pixel_value=255.0,
                p=1.0
            ),
            ToTensorV2(p=1.0),
        ])

In [ ]:
seed_everything(960630)

test_df = pd.DataFrame()
test_df['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
test_data = CasDataset(test_df, '../input/cassava-leaf-disease-classification/test_images/', transforms_test)

In [ ]:
test_loader = DataLoader(
                test_data, 
                batch_size=32,
                num_workers=8,
                shuffle=False,
                pin_memory=False,
            )

In [ ]:
test_preds = []
device = torch.device('cuda:0')
model = CasModel(model="resnet34").to(device)

for fold in range(5):
    model.load_state_dict(torch.load(f'../input/casresnet34/resnet34_f_{fold}.pth'))

    with torch.no_grad():
        for t in range(tta):
            test_preds += [inference_one_epoch(model, test_loader, device)]

In [ ]:
test_preds = np.mean(test_preds, axis=0) 
test_df['label'] = np.argmax(test_preds, axis=1)
test_df.head()

In [ ]:
test_df.to_csv('submission.csv', index=False)